## Reed-Muller符号  

codes.ReedMullerCode(GF, r, m)の説明  
- GF  
符号が定義される有限体  
- r  
符号の次数 r次多項式  
- m  
多項式に含まれる変数の数  

In [17]:
from math import comb

In [60]:
p = 2; m = 1
F = GF(p**m, name = 'a')
r = 1
variables = 3
GRM = codes.ReedMullerCode(F, r, variables)

### Reed-Muller符号の性質確認  

生成行列

In [27]:
GRM.generator_matrix()

[1 1 1 1 1 1 1 1]
[0 1 0 1 0 1 0 1]
[0 0 1 1 0 0 1 1]
[0 0 0 0 1 1 1 1]

パリティ検査行列

In [28]:
GRM.parity_check_matrix()

[1 0 0 1 0 1 1 0]
[0 1 0 1 0 1 0 1]
[0 0 1 1 0 0 1 1]
[0 0 0 0 1 1 1 1]

符号長

In [29]:
GRM.length()

8

符号の次元

In [30]:
def reed_muller_dim(r, m):
    res = 0
    for i in range(r+1):
        res += comb(m, i)
    return res

In [31]:
reed_muller_dim(r, variables)

4

In [32]:
GRM.dimension()

4

最小距離

In [33]:
GRM.minimum_distance()

4

符号語

In [34]:
list(GRM)

[(0, 0, 0, 0, 0, 0, 0, 0),
 (1, 1, 1, 1, 1, 1, 1, 1),
 (0, 1, 0, 1, 0, 1, 0, 1),
 (1, 0, 1, 0, 1, 0, 1, 0),
 (0, 0, 1, 1, 0, 0, 1, 1),
 (1, 1, 0, 0, 1, 1, 0, 0),
 (0, 1, 1, 0, 0, 1, 1, 0),
 (1, 0, 0, 1, 1, 0, 0, 1),
 (0, 0, 0, 0, 1, 1, 1, 1),
 (1, 1, 1, 1, 0, 0, 0, 0),
 (0, 1, 0, 1, 1, 0, 1, 0),
 (1, 0, 1, 0, 0, 1, 0, 1),
 (0, 0, 1, 1, 1, 1, 0, 0),
 (1, 1, 0, 0, 0, 0, 1, 1),
 (0, 1, 1, 0, 1, 0, 0, 1),
 (1, 0, 0, 1, 0, 1, 1, 0)]

双対符号

In [35]:
GRM.dual_code()

[8, 4] linear code over GF(2)

### ブロックの構成  

In [61]:
blocks = []
for code in list(GRM):
    block = [i for i, bit in enumerate(code) if bit == 1]
    blocks.append(block)
blocks

[[],
 [0, 1, 2, 3, 4, 5, 6, 7],
 [1, 3, 5, 7],
 [0, 2, 4, 6],
 [2, 3, 6, 7],
 [0, 1, 4, 5],
 [1, 2, 5, 6],
 [0, 3, 4, 7],
 [4, 5, 6, 7],
 [0, 1, 2, 3],
 [1, 3, 4, 6],
 [0, 2, 5, 7],
 [2, 3, 4, 5],
 [0, 1, 6, 7],
 [1, 2, 4, 7],
 [0, 3, 5, 6]]

### Rank metricへの拡張  

In [43]:
def codeword_to_matrix(codeword, m):
    expanded = []
    for element in codeword:
        poly = element.polynomial()
        coeffs = poly.coefficients(sparse=False)
        coeffs += [0] * (m - len(coeffs))
        expanded.append(coeffs)
    flat_list = [coeff for vec in expanded for coeff in vec]
    return Matrix(GF(p), len(codeword), m, flat_list)

In [47]:
matrix_codewords = []
for codeword in GRM:
    matrix = codeword_to_matrix(codeword, m)
    matrix_codewords.append(matrix)

[
[0]  [1]  [0]  [1]  [0]  [1]  [0]  [1]  [0]  [1]  [0]  [1]  [0]  [1]
[0]  [1]  [1]  [0]  [0]  [1]  [1]  [0]  [0]  [1]  [1]  [0]  [0]  [1]
[0]  [1]  [0]  [1]  [1]  [0]  [1]  [0]  [0]  [1]  [0]  [1]  [1]  [0]
[0]  [1]  [1]  [0]  [1]  [0]  [0]  [1]  [0]  [1]  [1]  [0]  [1]  [0]
[0]  [1]  [0]  [1]  [0]  [1]  [0]  [1]  [1]  [0]  [1]  [0]  [1]  [0]
[0]  [1]  [1]  [0]  [0]  [1]  [1]  [0]  [1]  [0]  [0]  [1]  [1]  [0]
[0]  [1]  [0]  [1]  [1]  [0]  [1]  [0]  [1]  [0]  [1]  [0]  [0]  [1]
[0], [1], [1], [0], [1], [0], [0], [1], [1], [0], [0], [1], [0], [1],

[0]  [1]
[1]  [0]
[1]  [0]
[0]  [1]
[1]  [0]
[0]  [1]
[0]  [1]
[1], [0]
]

In [45]:
for i in range(len(matrix_codewords)):
    for j in range(i + 1, len(matrix_codewords)):
        rank_distance = (matrix_codewords[i] - matrix_codewords[j]).rank()
        M = matrix_codewords[i] + matrix_codewords[j]
        if rank_distance == 0:
            print(f"符号語 {i+1} と {j+1} のランク距離が0")
        if M not in matrix_codewords:
            print(f"符号語 {i+1} と {j+1} が演算に閉じていない")

In [46]:
min_rank_distance = float('inf')
for i in range(len(matrix_codewords)):
    for j in range(i + 1, len(matrix_codewords)):
        rank_distance = (matrix_codewords[i] - matrix_codewords[j]).rank()
        min_rank_distance = min(min_rank_distance, rank_distance)

print(f"符号の最小ランク距離: {min_rank_distance}")

符号の最小ランク距離: 1


## Reed-Muller符号からのブロックの構成  